In [ ]:
# Проверка коннекта с базой чтения
import pymssql


server = 'xx.xx.xx.xx' 
database = 'db_name' 
user = 'user' 
password = 'password'
conn = pymssql.connect(server, user, password, database)
cursor = conn.cursor()
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

In [ ]:
# Проверка коннекта с базой записи
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT


server = 'xx.xx.xx.xx' 
database = 'db_name' 
user = 'user' 
password = 'password'
conn = psycopg2.connect(host=server, user=user, password=password, database=database)
cursor = conn.cursor()
cursor.execute("SELECT version();") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

   # feedmaker
   
    Скрипт feedmaker предназначен для считывания информации о комплектах из Excel-файла, получение информации о товарах, входящих в комплект из базы данных, получения изображений компонентов комплекта, формирования главного изображения комплектов и формирования xml-фида, содержащего полную информацию о комплектах, сохранение информации о комплектах в базе данных.
    Формат Excel-файла в img_1. Колонка "id категории" должна быть обязательно. Если ее нет, или она имеет иное, чем здесь указано название скрипт прекращает работу. Названия листов, подготовленных к обработке должны содержать слова "комплект" и "готово" в любом сочетании. Имя входного Excel-файла должно быть указано в константе "XLS_FILE".
    Скрипт предназначен для запуска в среде jupyter notebook. Разработка велась на версии jupyter 6.0.1 одновременно под Windows и под Linux Mint (особенности выявлены при подключении шрифта: в Linux при подключении шрифта нужно указывать полный путь, это учтено в коде). Перед запуском необходимо проверить наличие требуемых пакетов (см. импорты), при отсутствии недостающие необходимо установить. Порядок установки определяется документацией на пакет. Установка пакета должна производиться в то же виртуальное окружение, в котором будет запусаться скрипт. Версия пакета Pillow должна быть 8.3.1 или выше, иначе выравнивание текстового блока на главном изображении комплекта работать не будет.
    Скрипт имеет два режима запуска: DEBUG и PROD. Режим устанавливается в секции констант путем присвоения константе MODE значения одной из констант DEBUG или PROD. Если константе MODE не присвоено ни одно из значений, скрипт завершает работу сообщением об ошибке. Режимы оличаются следующим:
    1) в режиме DEBUG catalog/images/ размещается в текущей папке скрипта, а в режиме PROD здесь C:/inetpub/wwwroot/export/ozon/catalog/images/;
    2) в режиме DEBUG из входного Excel-файла обрабатывается только один лист с комплектами из входного Excel-файла, а в режиме PROD обрабатываются все листы, удовлетворяющие указанным выше условиям.
    Блок констант по сути является параметрами скрипта, изменяя которые можно в определенных пределах менять его поведение. В частности, задавать имена и пути к входному и выходному файлам, задавать параметры подключения в базе данных, параметры главного фото комплекта и т.д.
    Порядок запуска:
    1) установить курсор в ячейке jupyter, содержащей скрипт (выбрать ячейку со скриптом);
    2) одновременно нажать клавиши `ctrl + enter`
    По ходу выполнения скрипт выдает информацию логирования с метками времени, по которой можно судить о выполнении тех или иных операций и скорости их выполнения. Успешное выполнение скрипта завершается выводом 'Работа скрипта успешно завершена.'
    
  ## Версия 0.3.0
    
    В версии 0.3.0 добавились следующие параметрв:
    Размер шрифта, px;
    Расстояние между строками, px;
    Дина строки, симв;
    Максимальное количество записей в фиде. 20000 записей дают фид весом около 100 Мб;
    Порог определения цветного фона от 0.0 до 1.0. Цветной фон определяется по отклонению СКО от Среднего по 4-м точкам в углах изображения, отстоящих на 10 px от краев.
    
    В версии 0.3.0 добавились следующие фичи:
    1. Фиды разбиваются на части на части, максимальный вес каждого не превышает 100 МБ.
    2. В описании артикула 2334309 отфильтрована фраза о скрапбукинге (она не относится к этой позиции) “Скрапбукинг давно стал трендом в рукоделии. Этот вид творчества поможет вам открыть в себе новые таланты и создать уникальные вещи. Сохраните милые сердцу воспоминания в альбоме, сделанном своими руками. Или сотворите для близкого человека трогательную открытку, выразив свои чувства и эмоции. </p> <p>Попробовав заняться скрапбукингом один раз, вы влюбитесь в него навсегда! А мы предложим вам огромный выбор качественных товаров по приятным ценам, например”.
    3. В названии слово набор сделано с большой буквы. Пример: Набор: мыльная основа, краситель гелевый, диоксид титана, пластиковая форма.
    4. На фото увеличены расстояние между строчками. Определен параметр, с помощью которого можно менять расстояние между строчками
    5. На фото добавлена обводка изображения  2 px. Пример http://joxi.ru/vAWBYk0C35v5V2
    6. Добавлены шаблоны для главного изображения согласно ТЗ https://docs.google.com/document/d/102fnHdw7GLq3j4JuoJqcE2Z1A4V67n7RxraPbj0rrNg/edit#heading=h.ylf1nbguytyu
    7. В описании каждого артикула добавлены теги <br>, чтобы обеспечить раздельно отображение абзацев, фраза “в комплекте ед.” тоже с нового абзаца.
    8. Добавлен механизм фильтрации стоп-брендов и стоп-слов. В БД есть 2 таблицы - запрещенные бренды и запрещенные слова. Родион напишет названия этих таблиц. Все формируемые комплекты нужно проверить - если среди брендов включаемых товаров есть запрещенный, то его не нужно включать в фид. Также по стоп-словам - если в названии товара есть запрещенное слово, то комплект с этим товаром также не нужно включать в фид. Слова проверяются по полному вхождению. Если там звездочка *, то это означает все символы до или после
    
## Версия 0.3.1

   Изменен алгоритм расчета габаритных размеров комплекта: сумма меньших  измерений (без привязки длинна, высота или ширина), остальные значения берем по максимальной величине  например, пенал 20х3х2  и альбом 25х15х2. Размер упаковки комплекта 25х15х4. <br><br>
   Добавлены вспомогательные функции pgdb_delete_all_kt() и kt_records_coun(), не участвующие в основом workflow. Данные функцмм предназначены для подсчета записей комплектов в определенных таблицах и для удаления этих записей при необходимости.
       
    Павел Попов
    12.07.2022 г.

In [ ]:
import asyncio
import math
import re
import sys
import pymssql
import os
import platform
import requests
import logging
import numpy as np
import xml.dom.minidom
import traceback
import psycopg2
import psycopg2.extras
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from openpyxl import load_workbook
# Pillow==8.3.1 Иначе не работает центрирование текста в картинке
from PIL import Image, ImageDraw, ImageFont, ImageOps
from datetime import datetime
from lxml import etree as ET
from datetime import datetime

"""
SCRIPT_NAME: 'feedmaker'

VERSION: '0.3.1'
"""

DEBUG = 0
PROD = 1

# Режимы запуска скрипта MODE может быть либо DEBUG (отлада) либо PROD (промэксплуатация)
MODE = PROD

# Название входного Excel-файла
XLS_FILE = 'Almazvysh-risov-epoksid_compl.xlsx'

XML_PATH = 'E:\\inetpub\\wwwroot\\export\\ozon\\catalog\\feeds'

# Название выходного файла фида без расширения
COMPLECT_LIST_XML = 'Almazvysh-risov-epoksid_compl'

# Таймаут интернет подключения при скачивании изображений
TIMEOUT = (15, 25)

# Параметры соединения с базой данных
CONNECTION_PARAMS = {
    'ip': 'xx.xx.xx.xx',
    'server': 'xx.xx.xx.xx',
    'database': 'db_name',
    'user': 'user',
    'password': 'password'
}

CONNECTION_SAVE_PARAMS = {
    'ip': 'xx.xx.xx.xx',
    'server': 'xx.xx.xx.xx',
    'port': 'pppp',
    'database': 'db_name',
    'user': 'user',
    'password': 'password'
}

# Список предлогов
PRETEXTS = ['для', 'с', 'на', 'в', 'от', 'у', 'к', 'за', 'c']

# Базовый url для ссылок на изображения
PHOTO_URL = 'http://141.101.204.99/export/ozon/catalog/images/'

# Корневой путь для записи изображений на диск и их считывания. В режиме DEBUG catalog/images/ размещается 
# в текущей папке скрипта, в режиме PROD /catalog/images/ размещается в корне файловой системы
if not MODE:
    PHOTO_PATH = 'catalog/images/'
else:
    PHOTO_PATH = 'E:/inetpub/wwwroot/export/ozon/catalog/images/'

# Параметры миниатюр на главном фото
Y_center_main = 665 # координата y центра круга главного товара
X_center_main =  465 # координата x центра круга главного товара
R_main = 290 # Радиус миниатюры главного товара
R_child = 95 # Радиус миниатюры дочернего товара
DELTA = 70 # Расстояние от центра дочернего изо до главной окружности
BORDER_main = 15 # Обводка главного товара в пикселях
BORDER_child = 10 # Обводка дополнительных товаров в пикселях
COLOR_main_border = '#8D9BA9' # Цвет обводки главного товара
COLOR_child_border = '#D9D9D9' # Цвет обводки дополнительного товара
X_size_main = 700 # Х-размер фото комплекта
Y_size_main = 900 # Y-размер фото комплекта
ANGLE_SECTOR = 160 # Угловой сектор в градусах
# Шрифт для текстового блока главного изображения комплекта
if platform.system() == 'Linux':
    FONT = '/usr/share/fonts/truetype/arial_bold.ttf'
else:
    FONT ="arialbd.ttf" # Такая форма работает в Windows
# Размер шрифта
FONT_SIZE = 26
# Расстояние между строками, px
STRING_HEIGHT = 10
# Дина строки
STRING_LENGTH = 25

# Максимальное количество записей в фиде. 20000 записей дают фид весом около 100 Мб
RECORDS_NUMBER = 15000

# Порог определения цветного фона от 0.0 до 1.0
COLOR_THRESHOLD = 0.2


loop = asyncio.get_event_loop()


logging.basicConfig(
    format="%(asctime)-15s [%(levelname)s] %(funcName)s: %(message)s",
    level=logging.INFO)


class Product():
    
    init_row = ['', '', '', '', '', 0, 0, 0, 0, 0]
    
    def __init__(self, sheet_row=init_row, color_name='', matherial='', puck_sizes='',
                 category_id='', quantity_in_set=1, quantity_in_puck=1, complect=[], files_link_list=None, country = '',
                 char_set = set()):
        self.product_id = sheet_row[1]
        self.artno = sheet_row[0]
        self.name = sheet_row[2]
        self.brand_name = sheet_row[3]
        self.description = re.sub(r'Скрапбукинг\D*\d*например', '', sheet_row[4])
        self.price = sheet_row[5]
        self.weight = sheet_row[6]
        self.length = sheet_row[7]
        self.width = sheet_row[8]
        self.height = sheet_row[9]
        self.color_name = color_name
        self.matherial = matherial
        self.puck_sizes = puck_sizes
        self.category_id = category_id
        self.quantity_in_set = quantity_in_set
        self.quantity_in_puck = quantity_in_puck
        self.complect = complect
        self.files_link_list = files_link_list
        self.country = country
        self.char_set = char_set
        
    def __str__(self):
        return (
            self.product_id + '; ' +
             self.artno + '; ' +
             self.name + '; ' +
             self.brand_name + '; ' +
             self.description + '; ' +
             str(self.price) + '; ' +
             str(self.weight) + '; ' +
             str(self.length) + '; ' +
             str(self.width) + '; ' +
             str(self.height) + '; ' +
             self.color_name + '; ' +
             self.matherial + '; ' +
             self.puck_sizes + '; ' +
             self.category_id + '; ' +
             str(self.quantity_in_set) + '; ' +
             str(self.quantity_in_puck) + '; ' +
             str(self.complect) + '; ' +
             str(self.files_link_list) + '; ' +
             str(self.country) + '; ' +
             str(self.char_set) + '; '
        )


class Product_photo():
    def __init__(self, sheet_row):
        self.product_id = sheet_row[1]
        self.artno = sheet_row[0]
        self.OriginName = sheet_row[2]
        self.main = sheet_row[3]
        

class Product_chars():
    def __init__(self, artno, product_id, chars):
        self.product_id = product_id
        self.artno = artno
        self.chars = chars


# Получить листы с готовыми комплектами.
# В названии листа должны быть слова "комплект" и "готово"
def get_complect_sheets(worbook):   
    if workbook is None:
        print('Входной Excel-файл поврежден или не существует')
        return None
    
    logging.info("Start")
    complects_sheets = [tab for tab in worbook if str(tab).lower().find('компл') != -1 and 
                           str(tab).lower().find('готово') != -1
                       ]
    logging.info("Finish")
    return complects_sheets


# Из листов с комплектами получаем список протокомплектов и список задействованных артикулов
def get_protocomplect_and_artno_lists(connection_params, complects_sheets):
    if complects_sheets == [] or complects_sheets is None:
        print('Excel-файл не содержит комплектов для обработки, поврежден или отсутсвует.')
        return None, None
    
    logging.info("Start")
    protocomplects_list = []
    artno_list = []
    column_counter = 0
    for sheet in complects_sheets:
        column_quantity = 0
        while str(sheet[1][column_quantity].value).lower().find('категор') == -1 and column_quantity != 100:
            column_quantity += 1
        column_counter = column_counter + column_quantity
        protocomplect_str_list = []
        if column_quantity != 100:
            for row in sheet.iter_rows(min_row=2, max_col=column_quantity+1):
                protocomplect = [str(int(cell.value)) for cell in row if cell.value]
                [artno_list.append(str(int(cell.value))) for cell in row if cell.value]
                if len(protocomplect) > 1:
                    protocomplect_str = str(protocomplect)
                    if protocomplect_str not in protocomplect_str_list:
                        protocomplect_str_list.append(protocomplect_str)
                        protocomplects_list.append(protocomplect)

    if column_counter == len(complects_sheets)*100:
        print('Excel - листы не соответствуют требуемому формату: отсутствует колонка "id категории"')
        return None, None
    
    try:
        conn = pymssql.connect(connection_params['server'], connection_params['user'],
                               connection_params['password'], connection_params['database'],
                               login_timeout=15, timeout=15)
        cursor = conn.cursor()
        cursor.execute(f"""SELECT p.artno
                                FROM catalog.product p 
                                WHERE p.artno IN {tuple(set(artno_list))};""")
        artno_list_db = []
        for row in cursor:
            artno_list_db.append(row[0])
    except Exception as error:
        print("Ошибка при работе с MSSQL", error)
    finally:
        if conn:
            conn.close()
    protocomplects_list_db = protocomplects_list.copy()
    for complect in protocomplects_list:            
        for artno in complect[:-1]:
            if artno not in artno_list_db:
                try:
                    protocomplects_list_db.remove(complect)
                except Exception as e:
                    pass
    logging.info("Finish")
    return protocomplects_list_db, tuple(set(artno_list_db))


# Основной запрос
def main_query(connection_params, artno_tuple):
    if artno_tuple is None:
        print('Пустой список артикулов')
        return None
    logging.info("Start")
    products_list = []
    try:
        conn = pymssql.connect(connection_params['server'], connection_params['user'],
                               connection_params['password'], connection_params['database'],
                               login_timeout=15, timeout=15)
        cursor = conn.cursor()
        cursor.execute(f"""SELECT p.artno,
                                p.ProductId,
                                p.Name,
                                b.BrandName,
                                p.Description,
                                o.Price,
                                o.Weight,
                                o.Length,
                                o.Width,
                                o.Height
                                FROM catalog.product p 
                                LEFT OUTER JOIN catalog.offer o ON  p.ProductId=o.ProductId 
                                LEFT OUTER JOIN catalog.Brand b ON  p.BrandID=b.BrandID
                                WHERE p.artno IN {artno_tuple};""")
        for row in cursor:
            product = Product(row)
            products_list.append(product)
    except Exception as error:
        print("Ошибка при работе с MSSQL", error)
    finally:
        if conn:
            conn.close()
    logging.info("Finish")
    return sorted(products_list, key=lambda x: x.artno)


# Все характеристики
def allchar_query(connection_params, artno_tuple):
    if artno_tuple is None:
        print('Пустой список артикулов')
        return None
    logging.info("Start")
    products_chars = []
    try:
        conn = pymssql.connect(connection_params['server'], connection_params['user'],
                               connection_params['password'], connection_params['database'],
                               login_timeout=15, timeout=15)
        cursor = conn.cursor()
        cursor.execute(f"""SELECT p.artno,
                                p.ProductId,
                                cp.Name,
                                pv.Value,
                                pv.PropertyValueID
                                FROM catalog.product p 
                                FULL OUTER JOIN catalog.ProductPropertyValue ppv ON ppv.ProductID=p.ProductId
                                FULL OUTER JOIN catalog.PropertyValue pv ON pv.PropertyValueID=ppv.PropertyValueID
                                FULL OUTER JOIN catalog.Property cp ON cp.PropertyID=pv.PropertyID
                                WHERE p.artno IN {artno_tuple}
                                ORDER BY p.ProductId;""")
        artno = None
        artno_chars = []
        for row in cursor:
            if row[0] != artno:
                arch = artno_chars
                if arch:
                    product_chars = Product_chars(arch[0], arch[1], arch[2])
                    products_chars.append(product_chars)
                artno_chars = []
                artno = row[0]
                artno_chars.extend([row[0], row[1], set()])
            if (row[2] != 'Размер упаковки' and
                row[2] != 'Вес, кг' and
                row[2] != 'Объём, мл' and
                row[2] != 'Склад' and
                row[2] != 'Набор' and
                row[2] != 'Длина' and
                row[2] != 'Ширина' and
                row[2] != 'Высота' and
                row[2] != 'Глубина' and
                row[2] != 'Форма выпуска'):
                artno_chars[2].add((row[2], row[3], row[4]))
        arch = artno_chars
        if arch:
            product_chars = Product_chars(arch[0], arch[1], arch[2])
            products_chars.append(product_chars)
    except Exception as error:
        print("Ошибка при работе с MSSQL", error)
    finally:
        if conn:
            conn.close()
    logging.info("Finish")
    return sorted(products_chars, key=lambda x: x.artno)


# Загрузка файла
async def get_file(url, directory, filename, fut):
    logging.info("Start")
    try:
        r = requests.get(url, timeout=TIMEOUT)
        with open(directory + filename, 'wb') as f:
            f.write(r.content)
            value = 1
    except Exception as e:
        print(e)
        value = 0
    finally:
        fut.set_result(value)
        logging.info("Finish")


# Исходные фотографии
def photo_query(connection_params, artno_tuple):
    if artno_tuple is None:
        print('Пустой список артикулов')
        return None
    logging.info("Start")
    products_photos = []
    try:
        conn = pymssql.connect(connection_params['server'], connection_params['user'],
                               connection_params['password'], connection_params['database'],
                               login_timeout=15, timeout=15)

        cursor = conn.cursor()
        cursor.execute(f"""SELECT p.artno,
                                p.ProductId,
                                ph.OriginName,
                                ph.main
                                FROM Catalog.Product p, catalog.Photo ph
                                WHERE p.artno IN {artno_tuple} and p.ProductId=ph.ObjId;""")

        for row in cursor:
            product_photo = Product_photo(row)
            products_photos.append(product_photo)
    except Exception as error:
        print("Ошибка при работе с MSSQL", error)
    finally:
        if conn:
            conn.close()
    logging.info("Finish")
    return sorted(products_photos,  key=lambda x: x.artno)


# Загрузка исходных фотографий
async def get_sources_photos(products_photos):
    if products_photos is None:
        print('Пустой список ссылок на фотографии товаров')
        return None
    
    logging.info("Start")
    processes = []
    photo_counter = 0
    fut_list = []
    for product_photo in products_photos:
        dir_artno = PHOTO_PATH + product_photo.artno + '/'
        if not os.path.exists(dir_artno):
            os.makedirs(dir_artno)
        if product_photo.main:
            i = 0
        else:
            i = 1
        file_path = PHOTO_PATH + product_photo.artno + '/'
        filename = product_photo.artno + '_' + str(i) + '.jpg'
        i += 1
        fut = loop.create_future()
        fut_list.append(fut)
        loop.create_task(get_file(product_photo.OriginName, dir_artno, filename, fut))

    print(f'Всего задач по загрузке фоографий {len(fut_list)}')
    
    for fut in fut_list:
        photo_counter = photo_counter + await fut
    print(f'Загрузка закончена. Загружено {photo_counter} фотографий')
    logging.info("Finish")


# Делаем круглую миниатюру
def make_round_image(image, width, height, border_color=None, border_width=None):
    logging.info("Start")
    
    orig_img = image
    
    orig_img = orig_img.resize((width, height), Image.ANTIALIAS)

    # getting height and width of 
    # an image using size() function
    height,width = orig_img.size

    # converting image to numpy array
    npImage = np.array(orig_img)

    # Creating mask image in L mode with same image size
    new_img = Image.new('L', orig_img.size, 0)

    # drawing on mask created image using Draw() function 
    draw = ImageDraw.Draw(new_img)

    # making circle on mask image using pieslice() function
    draw.pieslice([0,0,width,height],0,360,fill=255)

    # Converting the mask Image to numpy array
    np_new = np.array(new_img)

    # stack the array sequence
    # (original image array with mask image) depth wise
    npImage = np.dstack((npImage, np_new))

    # converting array to an image using fromarray() function
    final_img = Image.fromarray(npImage).convert("RGBA")

    logging.info("Finish")
    return final_img


# Рассчет координаты x от угла
def get_new_x(angle, add=0):
    return int(X_center_main - (R_main + DELTA + add)*math.cos(math.radians(angle)) - R_child)


# Рассчет координаты y от угла
def get_igreeck(angle, add=0):
    return int(Y_center_main - (R_main + DELTA +add)*math.sin(math.radians(angle)) - R_child)


# Создаем многострочный блок для картинки
def name_to_multistring(name):
    words = name.lower().replace(',', '').replace('.', '').split()
    words_1 = []
    words_1.append('Набор для творчества:')
    for word in words[1:]:
        try:
            index = words.index(word)
            s = word
            if word in PRETEXTS:
                s = word + ' ' + words[index + 1].strip()
            if words[index - 1] not in PRETEXTS:
                words_1.append(s)
        except Exception as e:
            pass
    len_str = 0
    # Счетчик количества строк. Начальное значение 2, т.к. учитываем первую и последнюю.
    string_counter = 2
    for word in words_1[1:-1]:
        len_str = len_str + len(word)
        index = words_1.index(word)
        if index%2 == 0:
            word = word.strip() + ' +'
            words_1[index] = word
        if len_str > STRING_LENGTH:
            words_1[index] = word + '\n'
            string_counter += 1
            len_str = 0
    return " ".join([word + '\n' if words_1.index(word) == 0 else word for word in words_1]), string_counter


# Делаем главное фото комплекта
def make_main_image(complect_list):
    global DELTA
    
    if complect_list is None:
        print('Список комплектов пустой')
        return None
    
    logging.info("Start")
    
    img_main_cache = {}
    img_product_cache = {}
    img_round_main_cache = {}
    img_round_product_cache = {}
    img_rectangle_main_cash = {}
    
    POINTS = [(10, 10), (690, 10), (690, 690), (10, 690)]
        
    # Серая подложка для главного товара
    background_main = Image.new("RGB", (2*R_main + 2*BORDER_main, 2*R_main + 2*BORDER_main), COLOR_main_border)
    background_main_product_img = make_round_image(background_main, 2*R_main + 2*BORDER_main, 2*R_main + 2*BORDER_main)
    
    # Серая подложка для товаров комплекта
    background_child = Image.new("RGB", (2*R_child + 2*BORDER_child, 2*R_child + 2*BORDER_child), COLOR_child_border)
    background_product_img = make_round_image(background_child, 2*R_child + 2*BORDER_child, 2*R_child + 2*BORDER_child)
    
    # Перебираем все комплекты
    for complect in complect_list:
        
        # Проверяем, есть ли уже изображение в кэшэ
        try:
            main_product_img = img_main_cache[complect.complect[0]]
            place = 'кэш'
        except Exception as e:
            # Читаем изображение главного товара и сохраняем в кэш
            try:
                main_product_img = Image.open(PHOTO_PATH + complect.complect[0] +'/' + complect.complect[0] + '_0.jpg')
                img_main_cache[complect.complect[0]] = main_product_img
                place = 'диск'
            except Exception as e:
                print(e)
                return 'При открытии изображения главного товара произошел сбой'
        
        # Проверяем наличие цветного фона у главного изображения
        pix = main_product_img.convert('RGB').load()
        rgb_arr = []
        for point in POINTS:
            try:
                rgb_arr.append(list(pix[point[0], point[1]]))
            except Exception as e:
                print(e)
                print(POINTS)
                print(point)
                print(point[0], '#######', point[1])
                print(rgb_arr)
                print(pix)
                print(pix[point[0], point[1]])
                print(place)
                print(PHOTO_PATH + complect.complect[0] +'/' + complect.complect[0] + '_0.jpg')
                print(complect.complect[0])

        a = np.array(rgb_arr)
        rgb_mean = np.mean(a, axis = 0)
        rgb_std = np.std(a, axis = 0)
        if (rgb_std[0]/rgb_mean[0] > COLOR_THRESHOLD or
            rgb_std[1]/rgb_mean[1] > COLOR_THRESHOLD or
            rgb_std[2]/rgb_mean[2] > COLOR_THRESHOLD):
            color_background = True
        else:
            color_background = False
        
        # Проверяем, есть ли каталог для комплекта
        dir_artno = PHOTO_PATH + complect.artno + '/'
        if not os.path.exists(dir_artno):
            os.makedirs(dir_artno)
        
        # Проверяем, есть ли уже миниатюра в кэшэ
        if color_background:
            pass
        else:
            try:
                main_product_img = img_round_main_cache[complect.complect[0]]
            except Exception as e:
                main_product_img = make_round_image(main_product_img, 2*R_main, 2*R_main)
                img_round_main_cache[complect.complect[0]] = main_product_img
        
        if color_background and len(complect.complect[:-1]) == 2:
            main_product_img = main_product_img.convert("RGBA")
            new_img = Image.new('RGBA', (X_size_main, Y_size_main), 0)
            new_img.paste(main_product_img, (0, 200), mask=main_product_img)
        elif color_background and len(complect.complect[:-1]) == 3:
            main_product_img = main_product_img.convert("RGBA")
            new_img = Image.new('RGBA', (X_size_main, Y_size_main + 210), 0)
            new_img.paste(main_product_img, (0, 200), mask=main_product_img)
        else :
            main_product_img = main_product_img.convert("RGBA")
            new_img = Image.new('RGBA', (X_size_main, Y_size_main), 0)
            new_img.paste(background_main_product_img, (X_center_main - R_main - BORDER_main, Y_center_main - R_main - BORDER_main))
            new_img.paste(main_product_img, (X_center_main - R_main, Y_center_main - R_main), mask=main_product_img)
                
        image_list = []
        for artno in complect.complect[1:-1]:
            # Проверяем, есть ли уже миниатюра в кэшэ
            try:
                product_img = img_product_cache[artno]
            except Exception as e:
                product_img = Image.open(PHOTO_PATH + artno +'/' + artno + '_0.jpg')
                img_product_cache[artno] = product_img         
                        
            # Проверяем, есть ли уже миниатюра в кэшэ
            try:
                product_img = img_round_product_cache[artno]
            except Exception as e:
                product_img = make_round_image(product_img, 2*R_child, 2*R_child)
                img_round_product_cache[artno] = product_img

            image_list.append(product_img)
        
        if color_background and len(complect.complect[:-1]) == 2:
            for image in image_list:
                x = int(X_size_main/3*2)
                y = int(Y_size_main/3*2)
                new_img.paste(background_product_img, (x, y), mask=background_product_img)
                new_img.paste(image, (x+BORDER_child,y+BORDER_child), mask=image)
        elif color_background and len(complect.complect[:-1]) == 3:
            dx = 0
            for image in image_list:
                x = 50 + dx
                y = 900
                new_img.paste(background_product_img, (x, y), mask=background_product_img)
                new_img.paste(image, (x+BORDER_child,y+BORDER_child), mask=image)
                dx = 400
            # Пишем "+" между миниатюрами
            plus_draw = ImageDraw.Draw(new_img)
            text = '+'
            font = ImageFont.truetype(FONT, size=200)
            plus_draw.multiline_text((350, 1000), text, align='center', anchor='mm', font=font, fill=COLOR_main_border)
            
        else :
            # Определяем углы
            add = 0
            len_image_list = len(image_list)
            delta_angle = ANGLE_SECTOR/len_image_list
            if len_image_list >= 3:
                angle = -15
            elif len_image_list == 2:
                angle = 0
            elif len_image_list <= 1:
                angle = 45
                add = 100
        
            # Вставляем миниатюры в основное изображение
            for image in image_list:
                x = get_new_x(angle, add)
                y = get_igreeck(angle, add)

                new_img.paste(background_product_img, (x,y), mask=background_product_img)
                new_img.paste(image, (x+BORDER_child,y+BORDER_child), mask=image)

                angle = angle + delta_angle

            # Пишем "+" между миниатюрами
            if len_image_list == 2:
                angle = ANGLE_SECTOR/3 - 15
                x = get_new_x(angle) + R_child -30
                y = get_igreeck(angle) + R_child + 30
                plus_draw = ImageDraw.Draw(new_img)
                text = '+'
                font = ImageFont.truetype(FONT, size=200)
                plus_draw.multiline_text((x, y), text, align='center', anchor='ms', font=font, fill=COLOR_main_border)
        
        # Делаем текстовый блок        
        idraw = ImageDraw.Draw(new_img)
        text, string_counter = name_to_multistring(complect.name)
        font = ImageFont.truetype(FONT, size=FONT_SIZE)
        idraw.multiline_text((X_size_main/2, 100),
                             text, align='center', anchor='mm',
                             font=font, fill="black", spacing=STRING_HEIGHT)
        idraw.line(((0,200), (X_size_main,200)), 'black', width=2)
        
        # Делаем обводку изображения
        if color_background and len(complect.complect[:-1]) == 3:
            idraw.line(((0, 0), (X_size_main, 0)), 'black', width=2)
            idraw.line(((X_size_main-2, 0), (X_size_main-2, Y_size_main + 210)), 'black', width=2)
            idraw.line(((0, 0), (0, Y_size_main + 210)), 'black', width=2)
            idraw.line(((0, Y_size_main-2 + 210), (X_size_main, Y_size_main-2 + 210)), 'black', width=2)
        else:
            idraw.line(((0, 0), (X_size_main, 0)), 'black', width=2)
            idraw.line(((X_size_main-2, 0), (X_size_main-2, Y_size_main)), 'black', width=2)
            idraw.line(((0, 0), (0, Y_size_main)), 'black', width=2)
            idraw.line(((0, Y_size_main-2), (X_size_main, Y_size_main-2)), 'black', width=2)

        # Замена альфа-канала на цвет фона с возможностью последующего сохранения в формате jpg
        new_img.load()
        background = Image.new("RGB", new_img.size, (255, 255, 255))
        background.paste(new_img, mask=new_img.split()[3]) # 3 is the alpha channel
                                        
        filename =dir_artno + complect.artno + '_0' + '.jpg'
        try:
            background.save(filename, 'JPEG', quality=100)
        except Exception as e:
            print(e)
    

    logging.info("Finish")


# Создаем список комплектов с полной информацией
def create_complect_list(protocomplects_list,
                         products_list,
                         products_photos,
                         products_chars):
    
    if (protocomplects_list is None or
        products_list is None or
        products_photos is None or
        products_chars is None):
        print('Один из требуемых списков входных параметров пустой')
        return None
    
    logging.info("start")
    
    complect_list = []
    
    products_artno = [product.artno for product in products_list]
    photos_artno = [product.artno for product in products_photos]
    chars_artno = [product.artno for product in products_chars]
    
    for protocomplect in protocomplects_list:
        complect = Product(category_id=protocomplect[-1:][0])
        complect.complect = protocomplect
        artno_dimensions_list = []
        if len(protocomplect) >= 3:
            i = 0
            for artno in protocomplect[:-1]:
                # Артикул - все артикулы через тире и к конце 'kt'
                complect.artno = complect.artno + artno + '-'
                # Наименование по алгоритму задания
                if products_list[products_artno.index(artno)].name.split()[1] in PRETEXTS:
                    complect.name = (complect.name + ' ' +
                        " ".join(products_list[products_artno.index(artno)].name.split()[:3]).replace(',', '') + ', '
                    )
                else:
                    complect.name = (complect.name + ' ' +
                        " ".join(products_list[products_artno.index(artno)].name.split()[:2]).replace(',', '') + ', '
                    )
                # Бренды - бренды артикулов через запятую
                complect.brand_name = complect.brand_name + str(products_list[products_artno.index(artno)].brand_name) + ', '
                # Вес - сумма весов всех артикулов
                complect.weight = complect.weight + products_list[products_artno.index(artno)].weight
                # Цена - сумма цен артикулов
                complect.price = complect.price + products_list[products_artno.index(artno)].price
                # Высота комплекта
                artno_dimension = [products_list[products_artno.index(artno)].length,
                                   products_list[products_artno.index(artno)].width,
                                   products_list[products_artno.index(artno)].height]
                complect.height = complect.height + artno_dimension.pop(artno_dimension.index(min(artno_dimension)))
                # Размеры артикулов
                artno_dimensions_list.append(artno_dimension)
                # Описание - соединение названийи и описаний всех артикулов
                complect.description = (complect.description +
                    products_list[products_artno.index(artno)].name + ' ' +
                    products_list[products_artno.index(artno)].description + '<br><br>'
                )
                # Характеристика "в наборе, шт." установлена по умолчанию в 1
                # Характеристика "в упаковке, шт." установлена по умолчанию в 1
                # Другие характеристики формируем из характеристик фртикулов
                chars = products_chars[chars_artno.index(artno)].chars
                complect.char_set = complect.char_set.union(chars)
            # Артикул окончательно
            complect.artno = complect.artno + 'kt'
            # ссылки на фотографии комплекта формируются в функции add_links_to_complect_list(complect_list)
            # Наименование окончательно
            complect.name = 'Набор: ' + complect.name
            complect.name = complect.name[:-2]
            # Ширина и длина
            complect.width = max([artno_dimension[0] for artno_dimension in artno_dimensions_list])
            complect.length = max([artno_dimension[1] for artno_dimension in artno_dimensions_list])
            # Размер упаковки
            complect.puck_sizes = ('{:.0f}'.format(complect.height/10) + ' см' + ' x ' +
                                   '{:.0f}'.format(complect.width/10) + ' см' + ' x ' +
                                   '{:.0f}'.format(complect.length/10) + ' см')
            # Описание
            complect.description = complect.description + f'В комплекте {len(protocomplect[:-1])} ед.'            
            # Вставляем готовый комплект в список
            complect_list.append(complect)

    logging.info("Finish")
    return complect_list
    

# Создаем xml-фид
def create_xml(connection_save_params, complect_list):
    
    if complect_list is None:
        print('Список комплектов пустой')
        return None
    
    logging.info("Start")
    
    try:      
        # Соединение с базой
        connection = psycopg2.connect(user=connection_save_params['user'],
                                      password=connection_save_params['password'],
                                      host=connection_save_params['server'],
                                      port=connection_save_params['port'],
                                      database=connection_save_params['database'])
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = connection.cursor()
        
        # Читаем стопслова
        sql = """SELECT "values" from "rs_stopwords";"""
        cursor.execute(sql)
        stop_words = [item[0].replace("*", '') for item in cursor.fetchall()]
        
        # Читаем стопбренды
        sql = """SELECT "values" from "rs_stopbrands";"""
        cursor.execute(sql)
        stop_brands_set = set([item[0].replace("*", '') for item in cursor.fetchall()])
        
    except (Exception, Error) as error:
        connection.rollback()
        print("Ошибка при работе с PostgreSQL", error)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("Соединение с PostgreSQL закрыто")
            
    for i in range(0, int(len(complect_list)/RECORDS_NUMBER  + 1)):
        
        feed_counter = 0

        root = ET.Element("yml_catalog", date=str(datetime.now()))
        shop = ET.SubElement(root, "shop")

        name = ET.SubElement(shop, "name")
        name.text = 'РусЭкспресс — интернет-гипермаркет'

        company = ET.SubElement(shop, "company")
        company.text = 'РусЭкспресс — интернет-гипермаркет'

        url = ET.SubElement(shop, "url")
        url.text = 'https://rusexpress.ru'

        currencies = ET.SubElement(shop, "currencies")
        currency = ET.SubElement(currencies, "currency", id='RUB', rate='1')

        offers = ET.SubElement(shop, "offers")
    
        for complect in complect_list[i*RECORDS_NUMBER:(i+1)*RECORDS_NUMBER]:
            
            # Проверяем наличие стоп-брендов
            brand_set = set([brand.strip() for brand in complect.brand_name.split(', ') if brand])
            is_stop_brands_set = stop_brands_set & brand_set
            
            # Проверяем наличие стоп-слов
            c_name = [word[:1].lower() + word[1:] for word in complect.name[1:].split()]
            c_name_str = ' '.join(c_name)
            is_stop_words = 0
            j = 0
            len_stop_words = len(stop_words)
            while is_stop_words == 0 and j != len_stop_words:
                is_stop_words = c_name_str.find(stop_words[j]) + 1
                j += 1

            if not is_stop_brands_set and not is_stop_words:
            
                offer = ET.SubElement(offers, "offer", id=complect.artno, available='true')

                url = ET.SubElement(offer, "url")
                url.text = 'https://rusexpress.ru/products/suvenir-polistoun-slavnaya-obezyanka-s-bogatstvom-miks-7h5-5h5-5-cm'

                price = ET.SubElement(offer, "price")
                if complect.price > 0:
                    price.text = '{:.2f}'.format(complect.price)
                else:
                    price.text = 'не указано'

                categoryId = ET.SubElement(offer, 'categoryId')
                categoryId.text = complect.category_id

                shop_sku = ET.SubElement(offer, 'shop-sku')
                shop_sku.text = complect.artno

                vendor = ET.SubElement(offer, 'vendor')
                vendor.text = 'нет бренда'

                manufacturer = ET.SubElement(offer, 'manufacturer')
                manufacturer.text = 'не указан'

                brand_list = tuple(brand_set)
                for brand in brand_list:
                    all_brands = ET.SubElement(offer, 'all_brands')
                    all_brands.text = brand.replace(',', '').strip()

                for link in complect.files_link_list:
                    picture = ET.SubElement(offer, 'picture')
                    picture.text = link

                name = ET.SubElement(offer, 'name')
                name.text = complect.name[:1] + " ".join(c_name)
                name.text = ET.CDATA(name.text)

                description = ET.SubElement(offer, 'description')
                description.text = complect.description
                description.text = ET.CDATA(description.text)

                vendorCode = ET.SubElement(offer, 'vendorCode')
                vendorCode.text = complect.artno

                weight = ET.SubElement(offer, 'weight')
                if complect.weight > 0:
                    weight.text = '{:.3f}'.format(complect.weight)
                else:
                    weight.text = 'не указано'

                param = ET.SubElement(offer, 'param', name="Высота, см")
                if complect.height > 0:
                    param.text = '{:.0f}'.format(complect.height/10)
                else:
                    param.text = 'не указано'

                param = ET.SubElement(offer, 'param', name="Ширина, см")
                if complect.height > 0:
                    param.text = '{:.0f}'.format(complect.width/10)
                else:
                    param.text = 'не указано'

                param = ET.SubElement(offer, 'param', name="Глубина, см")
                if complect.length > 0:
                    param.text = '{:.0f}'.format(complect.length/10)
                else:
                    param.text = 'не указано'

                param = ET.SubElement(offer, 'param', name="Срок годности в днях")
                param.text = '365'

                param = ET.SubElement(offer, 'param', name="Единиц в одном товаре")
                param.text = str(len(complect.complect[:-1]))

                param = ET.SubElement(offer, 'param', name="Название модели")
                param.text = complect.artno

                param = ET.SubElement(offer, 'param', name="Каталог ВБ")
                param.text = 'Да'

                char_list = sorted(list(complect.char_set), key=lambda x: x[0])
                for char in char_list:
                    param = ET.SubElement(offer, 'param', name=char[0])
                    param.text = char[1]

                param = ET.SubElement(offer, 'param', name="Размер", unit="RU")
                param.text = complect.puck_sizes

                dimensions = ET.SubElement(offer, 'dimensions')
                dimensions.text = '{:.0f}/{:.0f}/{:.0f}'.format(complect.height/10, complect.width/10, complect.length/10)
                
                feed_counter += 1
                
        dom = xml.dom.minidom.parseString(ET.tostring(root, encoding='unicode')) # or xml.dom.minidom.parse(fname)
        pretty_xml_as_string = dom.toprettyxml()
        
        # Проверяем, есть ли каталог для комплекта
        dir_feed = XML_PATH + '/'
        if not os.path.exists(dir_feed):
            os.makedirs(dir_feed)
        
        filename = dir_feed + COMPLECT_LIST_XML + '-' + str(i) + '.xml'
        with open(filename, "w", encoding="utf-8") as fh:
            fh.write(pretty_xml_as_string)
        
        print(f'В файл {filename} вошло {feed_counter} комплектов')
    
    logging.info("Finish")
    
    
def add_links_to_complect_list(complect_list):
    
    if complect_list is None:
        print('Список комплектов пустой')
        return None
    
    logging.info("Start")
    for complect in complect_list:
        complect.files_link_list = []
        dir_complect = PHOTO_PATH + complect.artno + '/'
        dir_complect_files = os.listdir(dir_complect)
        base_url = PHOTO_URL + complect.artno + '/'
        for file in dir_complect_files:
            link = base_url + file
            complect.files_link_list.append(link)
        for artno in complect.complect[:-1]:
            dir_artno = PHOTO_PATH + artno + '/'
            dir_artno_files = os.listdir(dir_artno)
            base_url = PHOTO_URL + artno + '/'
            for file in dir_artno_files:
                link = base_url + file
                complect.files_link_list.append(link)
    logging.info("Finish")
    return complect_list


def db_save(connection_save_params, complect_list):
    if complect_list is None:
        print('Список комплектов пустой')
        return None
    
    logging.info("Start")
        
    try:      
        # Соединение для записи
        connection = psycopg2.connect(user=connection_save_params['user'],
                                      password=connection_save_params['password'],
                                      host=connection_save_params['server'],
                                      port=connection_save_params['port'],
                                      database=connection_save_params['database'])
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = connection.cursor()
        
        # Читаем послеждние айдишники нужных таблиц
        sql = """SELECT max("Price"."ProductID") from "Price";"""
        cursor.execute(sql)
        last_product_id = int(cursor.fetchone()[0])
        print('last_product_id', last_product_id)
        
        sql = """SELECT max("PhotoId") from "Photo";"""
        cursor.execute(sql)
        last_photo_id = int(cursor.fetchone()[0])
        
        sql = """SELECT max("OfferID") from "Price";"""
        cursor.execute(sql)
        last_offer_id = int(cursor.fetchone()[0])
        
        product_artno_list = []
        artno_list = tuple([complect.artno for complect in complect_list])
        print(f'Всего комплектов {len(artno_list)}')
        sql = f"""SELECT "ArtNo" FROM "Product" WHERE "ArtNo" in {artno_list}"""
        cursor.execute(sql)
        if cursor:
            product_artno_list = [row[0] for row in cursor.fetchall()]
        print(f'Из них уже записано в базу {len(product_artno_list)}')
        delta_set = set(artno_list) - set(product_artno_list)
        print(f'Будет записано в базу {len(delta_set)}')
        complect_list_delta = [complect for complect in complect_list if complect.artno in delta_set]
        
        j = 0
        product_data = []
        photo_data = []
        cat_data = []
        price_data = []
        char_data = []
        char_cash = {}
        for complect in complect_list_delta:
            j += 1
            
            last_product_id += 1
            
            # Готовим product
            product_data.append((last_product_id,
                            complect.artno,
                            complect.name,
                            complect.description))
            
            # Готовим photo
            for link in complect.files_link_list:
                last_photo_id += 1
                if link.find('kt_0.jpg') != -1:
                    main = True
                else:
                    main = False               
                row_tuple = (
                    last_photo_id,
                    last_product_id,
                    link,
                    datetime.now(),
                    main,
                    link
                )
                photo_data.append(row_tuple)
                
            # Готовим категорию
            cat_data.append((complect.category_id,
                             last_product_id,
                             True))

            # Готовим price
            last_offer_id += 1
            price_data.append((last_offer_id,
                               last_product_id,
                               complect.price,
                               complect.artno,
                               True,
                               complect.length,
                               complect.width,
                               complect.height,
                               complect.weight))

            # Готовим характеристики
            char_list = sorted(list(complect.char_set), key=lambda x: x[0])
            for char in char_list:
                row_tuple = (last_product_id,
                             char[2])
                char_data.append(row_tuple)
            print(f'Подготовлен комплект {complect.artno}')
        
        if complect_list_delta:
            # Записываем в Product    
            sql = """INSERT INTO "Product" ("ProductId",
                                            "ArtNo",
                                            "Name",
                                            "Description") 
                                            VALUES(%s, %s, %s, %s)"""
            psycopg2.extras.execute_batch(cursor, sql, product_data)
            print('В базу записаны products')

            # Записываем в Photo    
            sql = """INSERT INTO "Photo" ("PhotoId",
                                          "ObjId",
                                          "PhotoName",
                                          "ModifiedDate",
                                          "Main",
                                          "OriginName") 
                                           VALUES(%s, %s, %s, %s, %s, %s)"""
            psycopg2.extras.execute_batch(cursor, sql, photo_data)
            print('В базу записаны photos')

            # Записываем категорию
            sql = """INSERT INTO "ProductCategories" ("CategoryID",
                                                      "ProductID",
                                                      "Main") 
                                                      VALUES(%s, %s, %s)"""
            psycopg2.extras.execute_batch(cursor, sql, cat_data)
            print('В базу записаны категории')

            # Записываем Price    
            sql = """INSERT INTO "Price" ("OfferID",
                                          "ProductID",
                                          "Price",
                                          "ArtNo",
                                          "Main",
                                          "Length",
                                          "Width",
                                          "Height",
                                          "Weight") 
                                          VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
            psycopg2.extras.execute_batch(cursor, sql, price_data)
            print('В базу записаны prices')

            # Записываем характеристики
            sql = """INSERT INTO "ProductPropertyValue" ("ProductID",
                                                         "PropertyValueID") 
                                                         VALUES(%s, %s)"""
            psycopg2.extras.execute_batch(cursor, sql, char_data)
            print('В базу записаны характеристики')
        
    except (Exception, Error) as error:
        connection.rollback()
        print("Ошибка при работе с PostgreSQL", error)
        traceback.print_exc(limit=None, file=None, chain=True)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("Соединение с PostgreSQL закрыто")
    
    logging.info("Finish")
    
# Удаление записей, содержащих информацию о комплектах в таблицах Price, Photo, PriductCategories,
# ProductPropertyValue и Product
def pgdb_delete_all_kt(connection_save_params):
    
    logging.info("Start")
        
    try:      
        # Соединение для записи
        connection = psycopg2.connect(user=connection_save_params['user'],
                                      password=connection_save_params['password'],
                                      host=connection_save_params['server'],
                                      port=connection_save_params['port'],
                                      database=connection_save_params['database'])
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = connection.cursor()
        
        # Удаляем комплекты из Price
        sql = """DELETE from "Price" WHERE "Price"."ProductID" IN
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        print('Price удалены')
        
        # Удаляем комплекты из Photo
        sql = f"""DELETE from "Photo" WHERE "Photo"."ObjId" IN 
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        print('Photo удалены')
        
        # Удаляем комплекты из PriductCategories
        sql = f"""DELETE from "ProductCategories" 
                  WHERE "ProductCategories"."ProductID" IN 
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        print('Categories удалены')
        
        # Удаляем комплекты из Property
        sql = f"""DELETE from "ProductPropertyValue" 
                  WHERE "ProductPropertyValue"."ProductID" IN 
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        print('Property удалены')
        
        # Удаляем комплекты из ProductId
        sql = """DELETE from "Product" WHERE "ArtNo" LIKE '%kt';"""
        cursor.execute(sql)
        print('Product удалены')

    except (Exception, Error) as error:
        connection.rollback()
        print("Ошибка при работе с PostgreSQL", error)
        traceback.print_exc(limit=None, file=None, chain=True)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("Соединение с PostgreSQL закрыто")
    
    logging.info("Finish")
    
# Подсчет количества записей, содержащих информацию о комплектах в таблицах Price, Photo, PriductCategories,
# ProductPropertyValue и Product
def kt_records_count(connection_save_params):
    
    logging.info("Start")
        
    try:      
        # Соединение для записи
        connection = psycopg2.connect(user=connection_save_params['user'],
                                      password=connection_save_params['password'],
                                      host=connection_save_params['server'],
                                      port=connection_save_params['port'],
                                      database=connection_save_params['database'])
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cursor = connection.cursor()
        
        # Читаем Price
        sql = """SELECT "Price"."ProductID" from "Price" WHERE "Price"."ProductID" IN
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        if cursor:
            price_product_id_list = [row[0] for row in cursor.fetchall()]
        print('Price', len(price_product_id_list))
        
        # Читаем Photo
        sql = f"""SELECT "Photo"."PhotoId" from "Photo" WHERE "Photo"."ObjId" IN 
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        if cursor:
            photo_id_list = [row[0] for row in cursor.fetchall()]
        print('Photo', len(photo_id_list))
        
        # Читаем PriductCategories
        sql = f"""SELECT "ProductCategories"."ProductID" from "ProductCategories" 
                  WHERE "ProductCategories"."ProductID" IN 
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        if cursor:
            categories_id_list = [row[0] for row in cursor.fetchall()]
        print('Catewgories', len(categories_id_list))
        
        # Читаем ProductPropertyValue
        sql = f"""SELECT "ProductPropertyValue"."ProductID" from "ProductPropertyValue" 
                  WHERE "ProductPropertyValue"."ProductID" IN 
                 (SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt');"""
        cursor.execute(sql)
        if cursor:
            property_id_list = [row[0] for row in cursor.fetchall()]
        print('Property', len(property_id_list))
        
        # Читаем Product
        sql = """SELECT "ProductId" from "Product" WHERE "ArtNo" LIKE '%kt';"""
        cursor.execute(sql)
        if cursor:
            product_id_list = [row[0] for row in cursor.fetchall()]
        print('Product', len(product_id_list))
        
    except (Exception, Error) as error:
        connection.rollback()
        print("Ошибка при работе с PostgreSQL", error)
        traceback.print_exc(limit=None, file=None, chain=True)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("Соединение с PostgreSQL закрыто")
    
    logging.info("Finish")
    
    
if __name__ == '__main__':
    
    if MODE == DEBUG or MODE == PROD:
        
        try:
            workbook = load_workbook(XLS_FILE)
        except Exception as e:
            print(e)
            workbook = None

        complects_sheets = get_complect_sheets(workbook)
        print(f'К обработке приняты следующие листы:\n{complects_sheets}')
                
        if not MODE:
            protocomplects_list, artno_tuple = get_protocomplect_and_artno_lists(CONNECTION_PARAMS, complects_sheets[:1])
        else:
            protocomplects_list, artno_tuple = get_protocomplect_and_artno_lists(CONNECTION_PARAMS, complects_sheets)
        
        products_chars = allchar_query(CONNECTION_PARAMS, artno_tuple)

        products_list = main_query(CONNECTION_PARAMS, artno_tuple)

        products_photos = photo_query(CONNECTION_PARAMS, artno_tuple)

        complect_list = create_complect_list(protocomplects_list,
                                             products_list,
                                             products_photos,
                                             products_chars)

        await get_sources_photos(products_photos)

        make_main_image(complect_list)

        complect_list = add_links_to_complect_list(complect_list)

        create_xml(CONNECTION_SAVE_PARAMS, complect_list)
        
        if MODE:
            db_save(CONNECTION_SAVE_PARAMS, complect_list)

        print(f'Работа скрипта успешно завершена. Обработано {len(complect_list)} комплектов')
    else:
        print('Режим запуска скрипта не установлен, см. документацию')